In [1]:
import requests
import json
import datetime
import time
import os
import numpy as np
FETCH_INTERVAL = 300 #seconds -- How often are we pulling data from the weather API?


https://openweathermap.org/history

Historical weather data

In [28]:
import requests
from urllib.parse import urlencode

# Set the OpenWeatherMap API endpoint and load the API key from the local environment (.env). 
openweathermap_url = "https://api.openweathermap.org/data/3.0/onecall/day_summary?"

with open('../keys_weather.json') as fi:
    credentials = json.load(fi)
openweathermap_token = credentials['api_key']

lat = 36.37
long = -86.84

params = {
    "lat": lat,
    "lon": long,
    # "type": "hour",
    "date": '2020-03-04',
    "appid": openweathermap_token
}

# Encode the parameters and join them to the URL
final_url = f"{openweathermap_url}?{urlencode(params)}"

print(final_url)

https://api.openweathermap.org/data/3.0/onecall/day_summary??lat=36.37&lon=-86.84&date=2020-03-04&appid=248e2ff04ca3f046ae82fedc8b5a2ed2


In [ ]:
https://api.openweathermap.org/data/3.0/onecall/day_summary?lat=39.099724&lon=-94.578331&date=2020-03-04&appid={API key}

In [20]:
import requests

# Set the OpenWeatherMap API endpoint and load the API key from the local environment (.env). 
openweathermap_url = "https://history.openweathermap.org/data/2.5/history/city"

lat = 36.37261
long = -86.840126

params = {"lat": lat, "lon": long, "type": "hour", "appid": openweathermap_token}

# Make the API request and get the JSON response
response = requests.get(openweathermap_url, params=params)
weather_data = response.json()
print(weather_data)

{'cod': 401, 'message': 'Invalid API key. Please see http://openweathermap.org/faq#error401 for more info.'}


In [10]:
weather_data

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}

In [ ]:

# "dt" is the timestamp of the weather snapshot and the number of seconds that have elapsed since January 1, 1970, at 00:00:00 UTC.
# So, format the timestamp as "%Y-%m-%d %H:%M:%S"
timestamp = weather_data["dt"]
dt_object = datetime.datetime.fromtimestamp(timestamp) 
dt_string = dt_object.strftime("%Y-%m-%d %H:%M:%S")

# As usual, the 'precip' has some 'special' attributes... OpenWeatherMap does not include this attribute if there is no rain. 
# In this case, we are after a full time-series, where no zero rain is wanted to have a non-null field.
# So, default to 0.0.
precip = 0.0
if "rain" in weather_data:
    precip = weather_data["rain"]["1h"]

# Format the weather data as a JSON object. 
# This is where we are cherry-picking the data we are interested in. 
# This pre-defines the starting schema used in Tinybird. May be better to cast a wider net and pare down what isn't 
# wanted later in the Pipes ;) 
data = {
    "timestamp": dt_string,
    "site_name": "Nashville",
    "temp_f": weather_data["main"]["temp"],
    "precip": precip,
    "humidity": weather_data["main"]["humidity"],
    "pressure": weather_data["main"]["pressure"],
    "wind_speed": weather_data["wind"]["speed"],
    "wind_dir": weather_data["wind"]["deg"],
    "clouds": weather_data["clouds"]["all"],
    "description": weather_data["weather"][0]["description"]
}